In [ ]:
%matplotlib.inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import nept

In [ ]:
def get_counts(spikes, edges, gaussian_std=None, n_gaussian_std=5):
    dt = np.median(np.diff(edges))

    if gaussian_std is not None:
        n_points = n_gaussian_std * gaussian_std * 2 / dt
        n_points = max(n_points, 1.0)
        if n_points % 2 == 0:
            n_points += 1
        if n_points > len(edges):
            raise ValueError("gaussian_std is too large for these times")
        gaussian_filter = signal.gaussian(n_points, gaussian_std/dt)
        gaussian_filter /= np.sum(gaussian_filter)

    counts = np.zeros((len(spikes), len(edges)-1))
    for idx, spiketrain in enumerate(spikes):
        counts[idx] = np.histogram(spiketrain.time, bins=edges)[0]
        if gaussian_std is not None and gaussian_std > dt:
            counts[idx] = np.convolve(counts[idx], gaussian_filter, mode='same')

    return nept.AnalogSignal(counts.T, edges[:-1])

In [ ]:
spikes = [nept.SpikeTrain([1., 1., 1., 5., 5., 7.])]

edges = [0, 2.5, 4, 5, 6, 10]
counts = get_counts(spikes, edges)

In [ ]:
counts.data, counts.time

In [ ]:
spikes = [nept.SpikeTrain([0.8, 1.1, 1.2, 1.2, 2.1, 3.1])]
position = nept.Position(np.array([1., 6.]), np.array([0., 4.]))

In [ ]:
def counts_sliding(spikes, position, n_dts, dt, gaussian_std=None, n_gaussian_std=5):   
    dt_edges = np.arange(position.time[0], position.time[-1], dt)
    square_filter = np.ones(n_dts)
#     square_filter = np.ones(n_dts) * (1 / n_dts)
    print(square_filter)
    
    if gaussian_std is not None:
        n_points = n_gaussian_std * gaussian_std * 2 / dt
        n_points = max(n_points, 1.0)
        if n_points % 2 == 0:
            n_points += 1
        gaussian_filter = signal.gaussian(n_points, gaussian_std/dt)
        gaussian_filter /= np.sum(gaussian_filter)
        print(gaussian_filter)
        
        filtered_spikes = []
        for spiketrain in spikes:
            filtered_spikes.append(nept.SpikeTrain(np.convolve(spiketrain.time, gaussian_filter, mode='same'), spiketrain.label))
            
    else:
        filtered_spikes = spikes
    
    counts = np.zeros((len(filtered_spikes), len(dt_edges)-1))
    for idx, spiketrain in enumerate(filtered_spikes):
        counts[idx] = np.convolve(np.histogram(spiketrain.time, bins=dt_edges)[0], square_filter, mode='same')
    
    return dt_edges, counts

In [ ]:
y, t = counts_sliding(spikes, position, n_dts=5, dt=0.5, gaussian_std=None)

In [ ]:
t

In [ ]:
y

In [ ]:
r = np.array([2., 2., 3., 1., 5., 2.])
np.convolve(r, np.ones(4), mode='same')